In [2]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import numpy as np
import pandas as pd

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/IDS")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
failinimi = 'WDI.csv'
data = pd.read_csv(failinimi, delimiter=";", decimal=',')

years = []
for i in range(1960,1981):
  years.append(str(i))
print(years)

#Drop years with sparse data
data = data.drop(data[data.Time.isin(years)].index)
#Drop also columns that has no mean to find features that impact CO2 emissions
data = data.drop(columns=['Time', 'Country Name'])
data = data.dropna()

['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980']


In [4]:
data

,Energy use (kg of oil equivalent per capita),GDP per capita growth (annual %),CO2 emissions (metric tons per capita),"Industry (including construction), value added (annual % growth)","Manufacturing, value added (annual % growth)",Urban population (% of total population),Merchandise imports (current US$),Merchandise exports (current US$),Population growth (annual %),Final consumption expenditure (annual % growth),Alternative and nuclear energy (% of total energy use),Fossil fuel energy consumption (% of total)
5551,1440.780224,-6.668514,3.600810,-10.663334,-12.070848,83.313000,9.430000e+09,9.143000e+09,1.571960,-3.583958,4.890517,90.017182
5554,4694.676425,1.711080,15.432501,3.564033,2.048696,85.700000,2.621500e+10,2.147700e+10,1.586853,3.546086,1.762525,92.775579
5555,2897.009398,-0.398620,7.416161,-1.946131,0.006903,65.364000,2.104300e+10,1.584100e+10,0.255018,1.423855,12.098725,83.963760
5559,104.861607,4.443148,0.097003,5.009258,4.318814,15.801000,2.699000e+09,7.910000e+08,2.636995,-2.036555,0.626875,32.712616
5564,353.462764,6.990661,0.112310,19.099072,-6.024097,28.010000,5.430000e+08,3.400000e+07,2.732265,9.115025,0.000000,7.291785
...,...,...,...,...,...,...,...,...,...,...,...,...
14748,3278.352078,2.133926,6.449699,2.830677,4.072303,73.908184,4.609270e+12,4.929050e+12,0.218022,1.873950,19.778166,69.886026
14751,4605.921389,1.760227,10.649120,2.161468,2.402055,80.320929,1.152510e+13,1.114970e+13,0.568071,2.250090,13.901704,79.081901
14769,6888.874114,1.912329,15.928306,1.547004,0.502192,81.633083,2.746360e+12,1.912660e+12,0.734794,3.234141,12.750786,81.502287
14771,4130.813646,1.754260,9.124481,2.105228,2.510732,80.000667,1.031390e+13,9.668150e+12,0.657284,2.390075,13.354083,79.676298


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3669 entries, 5551 to 14774
Data columns (total 12 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Energy use (kg of oil equivalent per capita)                      3669 non-null   float64
 1   GDP per capita growth (annual %)                                  3669 non-null   float64
 2   CO2 emissions (metric tons per capita)                            3669 non-null   float64
 3   Industry (including construction), value added (annual % growth)  3669 non-null   float64
 4   Manufacturing, value added (annual % growth)                      3669 non-null   float64
 5   Urban population (% of total population)                          3669 non-null   float64
 6   Merchandise imports (current US$)                                 3669 non-null   float64
 7   Merchandise exports (current 

In [6]:
#Checking, are there any Nan's in dataframe.
print(len(data) - data.count())
#data.isnull()

Energy use (kg of oil equivalent per capita)                        0
GDP per capita growth (annual %)                                    0
CO2 emissions (metric tons per capita)                              0
Industry (including construction), value added (annual % growth)    0
Manufacturing, value added (annual % growth)                        0
Urban population (% of total population)                            0
Merchandise imports (current US$)                                   0
Merchandise exports (current US$)                                   0
Population growth (annual %)                                        0
Final consumption expenditure (annual % growth)                     0
Alternative and nuclear energy (% of total energy use)              0
Fossil fuel energy consumption (% of total)                         0
dtype: int64


In [125]:
#cols = data.columns.drop('Country Name')
#data[cols] = data[cols].apply(pd.to_numeric)
#data.dtypes

In [139]:
#1-hot for column 'Country Name'
#onehot = pd.get_dummies(data['Country Name']) 
#data = data.join(onehot)
#data = data.drop(columns='Country Name')

In [7]:
cols = list(data.columns)
#data[cols] = pd.to_numeric(data[cols])
#columns.remove('CO2 emissions (metric tons per capita)')
print(cols)
print(len(cols))

['Energy use (kg of oil equivalent per capita)', 'GDP per capita growth (annual %)', 'CO2 emissions (metric tons per capita)', 'Industry (including construction), value added (annual % growth)', 'Manufacturing, value added (annual % growth)', 'Urban population (% of total population)', 'Merchandise imports (current US$)', 'Merchandise exports (current US$)', 'Population growth (annual %)', 'Final consumption expenditure (annual % growth)', 'Alternative and nuclear energy (% of total energy use)', 'Fossil fuel energy consumption (% of total)']
12


In [8]:
#Remove predicting type
cols.remove('CO2 emissions (metric tons per capita)')
print(cols)
print(len(cols))

['Energy use (kg of oil equivalent per capita)', 'GDP per capita growth (annual %)', 'Industry (including construction), value added (annual % growth)', 'Manufacturing, value added (annual % growth)', 'Urban population (% of total population)', 'Merchandise imports (current US$)', 'Merchandise exports (current US$)', 'Population growth (annual %)', 'Final consumption expenditure (annual % growth)', 'Alternative and nuclear energy (% of total energy use)', 'Fossil fuel energy consumption (% of total)']
11


In [9]:
data.head()

,Energy use (kg of oil equivalent per capita),GDP per capita growth (annual %),CO2 emissions (metric tons per capita),"Industry (including construction), value added (annual % growth)","Manufacturing, value added (annual % growth)",Urban population (% of total population),Merchandise imports (current US$),Merchandise exports (current US$),Population growth (annual %),Final consumption expenditure (annual % growth),Alternative and nuclear energy (% of total energy use),Fossil fuel energy consumption (% of total)
5551,1440.780224,-6.668514,3.600810,-10.663334,-12.070848,83.313,9.430000e+09,9.143000e+09,1.571960,-3.583958,4.890517,90.017182
5554,4694.676425,1.711080,15.432501,3.564033,2.048696,85.700,2.621500e+10,2.147700e+10,1.586853,3.546086,1.762525,92.775579
5555,2897.009398,-0.398620,7.416161,-1.946131,0.006903,65.364,2.104300e+10,1.584100e+10,0.255018,1.423855,12.098725,83.963760
5559,104.861607,4.443148,0.097003,5.009258,4.318814,15.801,2.699000e+09,7.910000e+08,2.636995,-2.036555,0.626875,32.712616
5564,353.462764,6.990661,0.112310,19.099072,-6.024097,28.010,5.430000e+08,3.400000e+07,2.732265,9.115025,0.000000,7.291785


In [10]:
#Recursive method to find combinations in list
def combinations(L):
  if len(L) == 0:
    return [[]]
  result = []
  for c in combinations(L[1:]):
    result += [c, c+[L[0]]]
  return result

c = combinations(['x','y','z'])[1:]
print(c)

[['x'], ['y'], ['y', 'x'], ['z'], ['z', 'x'], ['z', 'y'], ['z', 'y', 'x']]


In [11]:
#Model with LinearRegression to predict the combination of features 
#that gives the smallest mean absolute error 

X = data[cols]
y = data[['CO2 emissions (metric tons per capita)']]

features = combinations(cols)[1:]
best = 100
combi = ""
for f in features:
  X_train, X_test, y_train, y_test = train_test_split(X[f].values, y.values, test_size=0.3, random_state=0)
  model = linear_model.LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)

  result = np.mean(np.abs(y_test - y_pred))
  if result < best:
    best = result
    combi = f

print(combi)
print(result)


['Fossil fuel energy consumption (% of total)', 'Alternative and nuclear energy (% of total energy use)', 'Industry (including construction), value added (annual % growth)', 'GDP per capita growth (annual %)', 'Energy use (kg of oil equivalent per capita)']
1.0559951780266006
